## Данный ноутбук ищет упоминания о компаниях в сообщениях

In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
#from Levenshtein import distance

In [4]:
tickers = {
'gazp' : ['gazp', 'газпром',   'миллер', 'газик', 'нашевсе', 'северный поток', 'южный поток', 'сила сибири', \
          'ru0007661625'],
    
'sber' : ['sber', 'сбер', 'греф', 'ru0009029540'], # добавить названия активов
'vtbr' : ['vtb', 'костин', 'втб', 'внешторгбанк', 'народное ipo', 'ru000A0JP5V6'],
'rosn':  ['rosn', 'роснефт', 'сечин', 'ru000a0j2q06'],
'gmkn':  ['gmkn', 'норникел', 'nornickel', 'потанин', 'гмк', 'норк', 'интеррос', 'гамак', 'ru0007288411'],
'moex':  ['московская биржа', 'мосбирж', 'moex', 'москухн', 'ru000a0jr4a1'],
'alrs':  ['алрос', 'alrosa', 'alrs', 'алмаз', 'бриллиант', 'нюрб', 'ru0007252813'],
'lkoh':  ['lkoh', 'luk', 'алекперов', 'лукойл', 'лучок', 'лучк', 'ru0009024277'],
'sngs':  ['сургут', 'sngs', 'surgutneftega', 'кубышка', 'сурик', 'снгс', 'ru0008926258'],
'tatn':  ['татнефт', 'tatneft', 'tatn', 'маганов', 'ru0009033591'],
'yndx':  ['yndx', 'yandex', 'яндекс', 'волож'],
'aflt':  ['аэрофлот', 'aflt', 'добролет', 'аврор', 'российские авиалинии', 'полубояринов'],
'fees':  ['фск еэс', 'fees', 'феес', 'федеральные электросети', 'феск'],
'magn':  ['ммк', 'magn', 'магнитк', 'магнитогорск', 'магна'],
'mtss':  ['mts', 'мтс'],
'nvtk':  ['nvtk', 'новатэк', 'нова тэк', 'novatek', 'ямал спг', 'михельсон'],
'nlmk':  ['нлмк', 'nlmk', 'липецк', 'лисин', 'липк'],
'chmf':  ['северстал', 'chmf', 'severstal', 'севка', 'череповец', 'мордашов'],
'mgnt':  ['mgnt', 'магнит', 'magnit', 'марафон групп', 'галицкий', 'тандер'],
'plzl':  ['plzl', 'polyus', 'полюс', 'polyusgold', 'сухой лог'],
'qiwi':  ['qiwi', 'киви'],
'rasp':  ['rasp', 'распад'],
'lsrg':  ['лср', 'lsr', 'элэсэр'],
'mtlr':  ['мечел', 'mtlr', 'mechel', 'ru000a0jpv70'],
'phor':  ['фосагр', 'phor', 'phosagr', 'фосфат', 'dap'],
'rtkm':  ['rtkm', 'rostelecom', 'ростел', 'tele2', 'ртк', 'рткм', 'осеевский'],
'hydr':  ['русгидр', 'hydr', 'rushydr', 'гидра', 'ru000a0jpkh7'],
'afks':  ['афк', 'afks', 'afk sistem', 'ozon', 'евтушенков', 'систем'],
'trmk':  ['тмк', 'трубная', 'trmk', 'tmk', 'трмк'],
'upro':  ['юнипро', 'upro', 'unip', 'эон россия', 'березовская грэс'],
'akrn':  ['акрон', 'akrn', 'akron', 'азотные удобрения'],
'pikk':  ['пик', 'pik', 'мортон'],
'lnta':  ['лент', 'lent', 'lnta', 'tpg capital'],
'msng':  ['мосэнерг', 'msng', 'mosenerg', 'мося'],
'tgka':  ['тгк1', 'тгк-1', 'тгк 1', 'tgka', 'tgk-1', 'tgk1', 'tgk 1'],
'rual':  ['русал', 'rusal', 'rual', 'русский алюминий', 'краз'],
'trnf':  ['транснефт', 'trnf', 'нефтепровод дружба'],
'sibn':  ['газпромнефт', 'sibn', 'gazpromneft', 'sibneft', 'сибнефт', 'газнефт'],
'nmtp':  ['нмтп', 'nmtp', 'новороссийск', 'новопорт'],
'tanl':  ['tanl', 'tantal', 'тант'],
'iskj':  ['искч', 'институт стволовых клеток', 'стволовые клетки', 'стволовых клеток', 'iskj'],
'yakg':  ['ятэк', 'yakg', 'yakutsk fue', 'якутская топл', 'ru0007796819'],
'gtlc':  ['gtl', 'джитиэл', 'гтл'],
'gche':  ['черкизов', 'cherkizov', 'che'],
'rolo':  ['русолов', 'rolo', 'rusolov', 'ролово'],
'svav':  ['соллерс', 'svav', 'sollers', 'северстальавт', 'уаз'],
'arsa':  ['арсагер', 'ук арса', 'arsa', 'arsager'],
'krkn':  ['саратовский нпз', 'снпз', 'krkn', 'сарнпз'],
'selg':  ['селигдар', 'seligdar', 'selg'],
'prfn':  ['чзпсн', 'профнастил', 'настил', 'prfn'],
'gaza':  ['gaza', 'газ завод', 'газ авто', 'газель', 'горьковский'],
'kbtk':  ['kbtk', 'кбтк', 'кузбас', 'даэния холдинг'],
'nknc':  ['nknc', 'нижнекамск', 'нкнх', 'нефтехим'],
'aptk':  ['aptk', 'аптек', '36 и 6', '36.6'],
'amez':  ['amez', 'ашинский', 'аша', 'amet', 'амез', 'амет'],
'dasb':  ['dasb', 'даг'],
'irkt':  ['иркут', 'irkt', 'сухой', 'суперджет', 'sukhoi', 'ssj'],
'lsng':  ['ленэнерг', 'lsng', 'ленк', 'петербургские электросети'],
'rlmn':  ['ролл', 'ролман', 'rlmn', 'роля'],
'obuv':  ['обувь росси', 'obuv', 'westfalika', 'обувьрус'],
'abrd':  ['абрау', 'дюрсо', 'abrd', 'abrau', 'durso', 'русский винный дом', 'борис титов', 'ru000a0js5t7'],
'agro':  ['agro', 'русагро', 'us7496552057'],
'albk':  ['бест эффортс', 'albk', 'best efforts', 'алор банкбестэфб', 'ru000a0js2m9'],
'alnu':  ['alnu', 'nyurba', 'алроса-нюрба', 'якуталмаз', 'ru000a0jp468'],
'aqua':  ['аквакультураaqua', 'aquacultur', 'русаква', 'русское море', 'ru000a0jqts3'],
'assb':  ['астрэнсб', 'assb', 'astrakhan', 'аэск', 'астраханэнергосбыт', 'ru000a0d8mm8'],
'avan':  ['avangard', 'авангард', 'ru000a0dm7b3'],
'bane':  ['башнефт', 'bane', 'bashneft', 'ru0007976957'],
'belu':  ['белуг', 'belu', 'belug', 'ru000a0hl5m1'],
'bisv':  ['bisv', 'bashinformsvyaz', 'bashtel', 'ru0009059216'],
'blng':  ['белон', 'blng', 'belon', 'ru000a0j2qg8'],
'brzl':  ['бурзолот', 'brzl', 'buryatzolot', 'бурятзолот', 'ru0009288658'],
'bspb':  ['бсп', 'bspb', 'bank st petersburg', 'банк санкт-петербург', 'ленбанк', 'ru0009100945'],
'cbom':  ['мкб', 'cbom', 'credit bank of moscow', 'московский кредитный банк', 'ru000a0jug31'],
'chep':  ['чтпз', 'chep', 'chelyabinsk pipe plant', 'chelpipe', 'chtpz', 'челябинский трубопрокатн', 'ru0009066807'],
'chgz':  ['рн-запсиб', 'рн запсиб', 'chgz', 'rn-western siberia', 'рн-западная сибирь', 'буй', 'ru0009082291'],
'chkz':  ['чкпз', 'chkz', 'chkpz', 'челябинский кузнечно-прессовый заво', 'chelyabinsk forge & press plant', \
          'гартунг', 'ru000a0jnst0'],
'cntl':  ['телеграф', 'cntl', 'telegraf', 'ru0007665147'],
'diod':  ['заводдиод', 'diod', 'диод', 'ru000a0jqwc1'],
'dsky':  ['dsky', 'detskiy mir', 'детский', 'ru000a0jsq90'],
'dvec':  ['дэк', 'dvec', 'feec', 'far-eastern energy', 'дальневосточная энергетическая компания', 'ru000a0jp2w1'],
'dzrd':  ['донскзр', 'dzrd', 'donskoy factory of radiocomponents', 'донской завод радиодеталей', \
          'donskoy zavod radiodetaley', 'ru000a0jqw11'],
'eelt':  ['евроэлтех', 'eelt', 'european eltech', 'европэлектротехника', 'европейская электротехника', 'каленков', \
          'ru000a0jww54'],
'eltz':  ['eltz', 'electrozinc', 'электроцинк', 'ru000a0b66v1'],
'enpg':  ['эн+', 'enpg', 'en plus', 'en+', 'ru000a100k72'],
'enru':  ['энелрос', 'enru', 'enel', 'энел', 'ru000a0f5un3'],
'etln':  ['etln', 'etalon', 'эталон', 'us29760g1031'],
'fesh':  ['двмп', 'fesh', 'fesco', 'дв морское пароходство', 'far-eastern shipping co plc', 'ru0008992318'],
'five':  ['five', 'x5', 'пятерочк', 'шехтерман', 'перекрест', 'карусел', 'us98387e2054'],
'flot':  ['совкомфлот', 'flot', 'sovcomflot', 'современный коммерческий флот', 'ru000a0jxnu8'],
'gazc':  ['газкон', 'gazc', 'ru000a0jqg86'],
'gazs':  ['газсервис', 'газ-сервис', 'gazs', 'gaz-service', 'ru000a0jqg78'],
'gazt':  ['газтек', 'газ-тек', 'gazt', 'gaz-tek', 'ru000a0js199'],
'gema':  ['ммцб', 'gema', 'mmtsb', 'biomaterials processing and cryostorage', 'гема', \
          'обработки и криохранения биоматериалов', 'ru000a100gc7'],
'gltr':  ['gltr', 'глобалтранс', 'globaltrans', 'us37949e2046'],
'gtrk':  ['гтм', 'gtrk', 'globaltruck', 'глобалтрак', 'gtm', 'ru000a0zyd22'],
'gtss':  ['геотек', 'gtss', 'geotech', 'сейсморазведка', 'ru000a0jq7u3'],
'hals':  ['галс', 'hals', 'пучков', 'ru000a0jnp96'],
'himc':  ['химпром', 'himc', 'khimprom', 'ru0009098990'],
'igst':  ['ижстал', 'igst', 'izhstal', 'козеннов', 'ru0002155292'],
'irao':  ['irao', 'inter rao', 'борис ковальчук', 'ru000a0jpnm1', 'рао', 'интер'],
'irgz':  ['иркэнерго', 'irgz', 'irkutskenergo', 'иркутскэнерго', 'ru0008960828'],
'jnos':  ['славн-янос', 'jnos', 'slavneft yaroslavnefteorgsintez', 'славнефт', 'ярославнефт', 'ru0009100895'],
'kazt':  ['куйбазот', 'kazt', 'kuibyshevazot', 'куйбышевазот', 'ru000a0b9bv2'],
'kbsb':  ['тнсэкубань', 'kbsb', 'tns energo kuban', 'энерго кубань', 'кубанская энергосбытовая компания', 'ru000a0jnj11'],
'kche':  ['камчатскэ', 'kche', 'kamchatskenergo', 'ru0006753498'],
'kgkc':  ['кургангк', 'kgkc', 'kurgan generation', 'курганская генер', 'кгк', 'ru000a0jp120'],
'klsb':  ['калужскск', 'klsb', 'kaluga power', 'калужская сбыт', 'ru000a0dkzk3'],
'kmaz':  ['камаз', 'kmaz', 'kamaz', 'ru0008959580'],
'kmez':  ['кмз', 'kmez', 'kovrov', 'ковровск', 'ru0006753613'],
'kmtz':  ['косогмз', 'kmtz', 'kosaya gora', 'косогорск', 'kosogorski', 'ru000a0hf1l0'],
'kogk':  ['коршгок', 'kogk', 'korshynov', 'коршуновск', 'ru0002155359'],
'krko':  ['ткзкк', 'krko', 'krasny kotelshchik', 'красный котельщик', 'ru0009098123'],
'krot':  ['красокт', 'krot', 'крот', 'krasnyj octyabr', 'красный октябрь', 'krasny oktyabr', 'ru0008913850'],
'krsb':  ['красэсб', 'krsb', 'krasnoyarskenergo', 'красноярскэнерго', 'ru000a0hmly1'],
'ksgr':  ['кокс', 'ksgr', 'koks', 'ru000a0ernt3'],
'ktsb':  ['ксб', 'ktsb', 'kostroma', 'костромск', ' kostromskaya sbyto', 'костромская сбытовая компания', 'ru000a0d8lw9'],
'kube':  ['кубанэн', 'kube', 'kubanenergo', 'кубаньэнерго', 'kuban power', 'rosseti kuban', 'ru0009046767'],
'kunf':  ['кузоцм', 'kunf', 'kuzocm', 'каменск-уральский', 'каменск уральский', 'ru000a0jpge2'],
'kuzb':  ['кузнецкий банк', 'kuzb', 'bank kuznetskiy', 'kuznetskiy bank', 'банк кузнец', 'ru000a0jsq66'],
'kzms':  ['кзмс', 'kzms', ' krasnokamsk metal', 'краснокамский завод', 'ru000a0jnhk2'],
'kzos':  ['органический синтез', 'kzos', 'оргсинтез', 'organicheskiy sintez', 'ru0009089825'],
'life':  ['фармсинтез', 'life', 'pharmsynt', 'ru000a0jr514'],
'lnzl':  ['лензолото', 'lnzl', 'lenzoloto', 'ленское золотодобывающ', 'ru000a0jp1n2'],
'lpsb':  ['лэск', 'lpsb', 'липецкая энерго', 'lipetsk power', 'lipetsk energ', 'ru000a0d8mr7'],
'mage':  ['магадэн', 'mage', 'magadanen', 'магаданэн', 'ru0006758919'],
'mail':  ['mail', 'mail.ru', 'mail', 'мэйл.ру', 'мэйл ру', 'добродеев', 'us5603172082'],
'mfgs':  ['мегион', 'mfgs', 'megionneftegas', 'ru0009011126'],
'mgnz':  ['смз', 'mgnz', 'smz', 'соликамский магн', 'ru0009100911'],
'mgts':  ['мгтс', 'mgts', 'moscow city tele', 'московская городская телефонная', 'moskovskaya gorodskaya tele',\
          'ru0009036461'],
'misb':  ['тнсэнмарэ', 'misb', 'tns energo mari', 'тнс энерго мари', 'ru000a0d8k33'],
'mobb':  ['mobb', 'moskovskii oblastnoi bank', 'московский областной банк', 'мособлбанк', 'ru000a0js7v9'],
'mori':  ['морион', 'mori', 'morion', 'телекоммуникационное оборудование', 'ru0009092431'],
'mrkc':  ['мрск центр', 'mrkc', 'ru000a0jppl8'],
'mrkk':  ['россети ск', 'mrkk', 'rosseti north caucasus', 'мрск северного кавказа', 'ru000a0jppq7'],
'mrkp':  ['мрск цп', 'mrkp', 'мрск центра и приволжья', 'ru000a0jpn96'],
'mrks':  ['mrks', 'rosseti siberia', 'россети сибирь', 'mrsk sibiri', 'мрск сибири', 'ru000a0jppf0'],
'mrku':  ['мрск ур', 'mrku', 'idgc of ural', 'ru000a0jppt1'],
'mrkv':  ['mrkv', 'rosseti volg', 'россети волг', 'мрск волг', 'ru000a0jppn4'],
'mrky':  ['россюг', 'mrky', 'rosseti south', 'россети юг', 'мрск юг', 'ru000a0jppg8'],
'mrkz':  ['мрск сз', 'mrkz', 'мрск северо-запад', 'mrsk severo zapad', 'ru000a0jppb9'],
'mrsb':  ['мордэнсб', 'mrsb', 'mordovia energ', 'мордовская энерг', 'ru000a0d9aj7'],
'msrs':  ['msrs', 'rosseti mosc', 'россети моск', 'московская объедин', 'ru000a0et7y7'],
'mstt':  ['мостотрест', 'mstt', 'mostotrest', 'ru0009177331'],
'mvid':  ['мвидео', 'mvid', 'м.видео', 'м видео', 'эльдорадо', 'изосимов', 'ru000a0jpga0'],
'nauk':  ['нпонаука', 'nauk', 'нпо наука', 'ru000a0jrpx9'],
'nfaz':  ['нефтекамск', 'nfaz', 'nefaz', 'ru0009115604'],
'nkhp':  ['нкхп', 'nkhp', 'novorossiysk grain', 'novorossiyskiy kombi', 'новороссийский комбин', 'ru000a0blwd7'],
'nksh':  ['нижкамшина', 'nksh', 'nizhnekamskshina', 'нижнекамскшина', 'ru0009100515'],
'nnsb':  ['тнсэннн', 'nnsb', 'tns energo n', 'тнс энерго н', 'ru000a0et5a1'],
'npof':  ['физик', 'npof', 'fizik', 'ru000a0jq5x1'],
'nsvz':  ['наукасвяз', 'nsvz', 'nauka-svyaz', 'nauka-tele', 'наука-связ', 'наука связ', 'ru000a0jqlb6'],
'ogkb':  ['огк-2', 'огк2', 'ogkb', 'ogk-2', 'семиколенов', 'ru000a0jng55'],
'paza':  ['павлавт', 'paza', 'pavlovo', 'павловский авт', 'pavlovskiy avt', 'паз', 'ru0009083240'],
'pmsb':  ['пермьэнсб', 'pmsb', 'perm energ', 'пермэнерг', 'permskaya energ', 'пермская энерг', 'ru000a0et123'],
'prmb':  ['prmb', 'primorie bank', 'приморье банк', 'банк приморье', 'ru000a0dpnq5'],
'prtk':  ['протек', 'prtk', 'protek', 'ru000a0jqu47'],
'rbcm':  ['рбк', 'rbcm', 'rbk', 'rbc', 'ru000a0jr6a6'],
'rdrb':  ['рдбанк', 'rdrb', 'rosdorbank', 'российскийдор', 'российский дор', 'ru000a0ju6q7'],
'rgss':  ['ргс', 'rgs', 'rosgosstra', 'росгосстра', 'гальперин', 'ru0008010855'],
'rkke':  ['энергияркк', 'rkke', 'spc energia', 'ркк энергия', 'ru0009095939'],
'rnft':  ['rnft', 'russneft', 'русснефт', 'ru000a0jse60'],
'rosb':  ['росбанк', 'rosb', 'rosbank', 'ru000a0hhk26'],
'rsti':  ['россет', 'rsti', 'российские сети', 'ru000a0jpvj0'],
'rtgz':  ['rtgz', 'rostovoblgaz', 'ростовоблгаз', 'ru000a0jpg04'],
'rtsb':  ['rtsb', 'tns energo rost', 'тнс энерго рост', 'ru000a0d8pb4'],
'rugr':  ['русгрэйн', 'ругрэйн', 'rugr', 'rusgrain', 'русское зерн', 'ru000a0jpnp4'],
'rusi':  ['икрусс', 'rusi', 'руссинвест', 'рус инвест', 'ru000a0jq9w5'],
'rusp':  ['русполимет', 'rusp', 'ruspol', 'ru000a0jnh21'],
'rzsb':  ['rzsb', 'ryazan energ', 'рязанская энерг', 'рязаньэнерго', 'ru000a0d9af5'],
'sago':  ['sago', 'samaraen', 'самараэнерго', 'самарэнерго', 'ru0009098255'],
'sare':  ['sare', 'saratoven', 'саратовэнерго', 'ru0009100754'],
'sfin':  ['сафмар', 'sfin', 'safmar', 'европлан', 'ru000a0jvw89'],
'slen':  ['сахэнер', 'slen', 'sakhalinen', 'сахалинэн', 'ru0009280465'],
'smlt':  ['самолет', 'smlt', 'samolet', 'ru000a0zzg02'],
'stsb':  ['stsb', 'stavropolen', 'ставрополэн', 'ставропольэн', 'ru000a0et1w4'],
'tasb':  ['tasb', 'tambov en', 'тамбовская эн', 'тамбовэнерго', 'ru000a0d8ng8'],
'tcsg':  ['tcs', 'tinkof', 'тисиэс', 'тинек', 'тинько', 'us87238u2033'],
'tgkb':  ['тгк 2', 'тгк-2', 'тгк2', 'tgkb', 'tgc-2', 'tgc2', 'tgc 2', 'ru000a0jngs7'],
'tgkd':  ['квадр', 'tgkd', 'quadra', 'ru000a0jnmz0'],
'tgkn':  ['тгк-14', 'тгк14', 'тгк 14', 'tgkn', 'tgk-14', 'tgk14', 'tgk 14', 'ru000a0h1es3'],
'tnse':  ['тнсэнерг', 'tns', 'tns energ', 'тнс энерг', 'ru000a0jucq5'],
'tors':  ['трк', 'tors', 'tomsk distribution', 'томская распред', 'ru000a0etzf2'],
'trcn':  ['транск', 'trcn', 'transcontainer', 'трансконтейнер', 'транс контейнер', 'ru000a0jprx9'],
'trfm':  ['трансф', 'trfm', 'transfin', 'трансфин', 'ru000a0jugp8'],
'ttlk':  ['таттел', 'ttlk', 'tattel', 'ru000a0hm5c1'],
'tuza':  ['тза', 'tuza', 'tuimazinsk', 'туймазинск', 'teuza', 'ru000a0hl7a2'],
'ucss':  ['окс', 'ucss', 'united credit', 'объединенные кредитн', 'ru000a0jtqk0'],
'ukuz':  ['южкузб', 'ukuz', 'southern kuzb', 'южный куз', 'юж куз', 'ru0005294775'],
'unac':  ['авиасткао', 'unac', 'united aircraft', 'объединенная авиа', 'ru000a0jplz7'],
'unkl':  ['юунк', 'unkl', 'southern ural', 'южно-уральский ник', 'ru0004887991'],
'urka':  ['урка', 'urka', 'uralka', 'уралка'],
'urkz':  ['уркузн', 'urkz', 'urals stampings', 'уралкуз', 'уральская кузниц', 'ru000a0jpfy2'],
'usbn':  ['уралсиб', 'usbn', 'uralsib', 'урсиб', 'ru0006929536'],
'utar':  ['ютэйр', 'utar', 'utair', 'ru0007661385'],
'uwgn':  ['овк', 'uwgn', 'united wagon', 'нпк овк', 'ru000a0jvbt9'],
'vgsb':  ['vgsb', 'volgograden', 'волгоградэн', 'volgograd en', 'ru000a0d8l73'],
'vjgz':  ['варьеган', 'vjgz', 'varyogan', 'varyegan', 'ru0009046460'],
'vlhz':  ['вхз', 'vlhz', 'vladimir chem', 'владимирский хим', 'ru0007984761'],
'vrsb':  ['vrsb', 'tns energo voronez', 'тнс энерго воронеж', 'ru000a0dpg67'],
'vsmo':  ['всмпо-авсм', 'vsmo', 'vsmpo-avisma', 'ависма', 'ru0009100291'],
'vsyd':  ['выбсуд', 'vsyd', 'vyborg ship', 'vyborgskiy sud', 'выборгский суд', 'ru0006334372'],
'vzrz':  ['возрожд', 'vzrz', 'vozrozhd', 'ru0009084214'],
'wtcm':  ['цмт', 'wtcm', 'world trade cen', 'центр международной торговли', 'ru0008137070'],
'yken':  ['якутскэнерг', 'yken', 'yakutskenerg', 'ru0009257075'],
'yrsb':  ['yrsb', 'tns energo yarosavl', 'тнс энерго ярославл', 'ru000a0d8760'],
'zill':  ['зил', 'zill', 'likhach', 'лихачев', 'ru0009086193'],
'zvez':  ['звезд', 'zvez', 'zvezd', 'ru0009091300']
}

In [9]:
pd.DataFrame.from_dict(tickers, orient='index').to_excel('tickers.xlsx')

In [3]:
trans = {'magnit': 'mgnt',
 'mts': 'mtss',
 'nornikel': 'gmkn',
 'surgutneftegas': 'sngs',
 'sberbank': 'sber',
 'polyus': 'plzl',
 'severstal': 'chmf',
 'novatek': 'nvtk',
 'moex': 'moex',
 'nlmk': 'nvtk',
 'afk sistema': 'afks',
 'yandex': 'yndx',
 'rosseti': 'rsti',
 'uralkaliy': 'urka',
 'unipro': 'upro',
 'transneft': 'trnf',
 'TGK-1': 'tgka',
 'rusgidro': 'hydr',
 'TMK': 'trmk',
 'rusolovo': 'rolo',
 'rostelekom': 'rtkm',
 'rusal': 'rual',
 'irkut': 'irkt',
 'gazpromneft': 'sibn',
 'dagsbyt': 'dasb',
 'cherkizovo': 'gche',
 'akron': 'akrn',
 'arsagera': 'arsa',
 'apteki_36_6': 'aptk',
 'ashinskiy_metzavod': 'amez',
 'fosagro': 'phor',
 'CHZPSN': 'prfn',
 'KTK': 'kbtk',
 'PIK': 'pikk',
 'NMTP': 'nmtp',
 'NKNH': 'nknc',
 'lsr': 'lsrg',
 'mosenergo': 'msng',
 'lenta': 'lnta',
 'gtl': 'gtlc',
 'YATEK': 'yakg',
 'vtb': 'vtbr',
 'saratovskiy_npz': 'krkn',
 'rosneft': 'rosn',
 'seligdar': 'selg',
 #'sib_gostinets': 'sibg',
 'GAZ': 'gaza',
 'fees': 'fees',
 'tatn': 'tatn',
 'sollers': 'svav',
 'QIWI': 'qiwi',
 'rollman': 'rlmn',
 'lukoil': 'lkoh',
 'lenergo': 'lsng',
 'ISKJ': 'iskj',
 'OBUV': 'obuv',
 'aeroflot': 'aflt',
 'mmk': 'magn',
 'gazprom': 'gazp',
 'mechel': 'mtlr',
 'alrosa': 'alrs',
 'raspadskaya': 'rasp',
 'tantal': 'tanl'}

In [66]:
count = 0
for i in tqdm(trans, position=0, leave=True):
    df = pd.read_csv(f'/is/texts/mfd/{i}_parsed.csv', sep='|')
    df = df[df.text!='DELETED']
    df = df.drop_duplicates('text')
    df = df[df.text.isna() != True]
    df['lower_text'] = df.text.str.lower()
    #переписать функцию
    #find_ticker(df, tickers[trans[i][0]], 'lower_text')
    df = df[['time', 'text', 'lower_text']]
    df.to_csv(f'/is/texts/mfd/edit/{trans[i]}.csv', sep='|', index=False)
    count += df.shape[0]
    print(f'{trans[i]} is done', end='\r')
#4308994

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


tanl is done


In [74]:
count = 0
for i in tqdm(os.listdir("/is/texts/mfd/edit/"), position=0, leave=True):
    df = pd.read_csv(f'/is/texts/mfd/edit/{i}', sep='|')
    find_ticker(df=df, ticker=i[:-4], tickers=tickers, field='lower_text')
    df = df[df[i[:-4]]==1]
    df.to_csv(f'/is/texts/mfd/tickers/{i}', sep='|', index=False)
    count += df.shape[0]
    print(f'{i[:-4]} is done', end='\r')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


tanl is done


In [2]:
tickers = [i for i in os.listdir("/is/texts/mfd/tickers/") if 'ipynb_checkpoints' not in i]

In [3]:
df = pd.DataFrame()
for i in tqdm(tickers, position=0, leave=True):
    df0 = pd.read_csv(f'/is/texts/mfd/tickers/{i}', sep='|')
    df0['ticker'] = i[:-4]
    df0 = df0[['time', 'text', 'lower_text', 'ticker']]
    df = pd.concat([df, df0])
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,time,text,lower_text,ticker
0,12.01.2013 18:48,"В четверг, 10 января, Магнит опубликовал предв...","в четверг, 10 января, магнит опубликовал предв...",mgnt
1,14.01.2013 18:56,Крупный портфельный инвестор в Россию — фонд O...,крупный портфельный инвестор в россию — фонд o...,mgnt
2,18.01.2013 22:22,"Ценовой шок, МММщики налетай, МММагнит продается","ценовой шок, мммщики налетай, мммагнит продается",mgnt
3,19.01.2013 14:59,"Хммм, Магнит будет расти и дальше. Нет ему ко...","хммм, магнит будет расти и дальше. нет ему ко...",mgnt
4,20.01.2013 01:09,"Резкого роста не будет,телегу медленно тянут в...","резкого роста не будет,телегу медленно тянут в...",mgnt


In [5]:
df.to_csv('/is/texts/mfd/mfd.csv', sep='|', index=False)

### Работа с датой

In [3]:
df = pd.read_csv('/is/texts/mfd/mfd.csv', sep='|')
df.head()

,time,text,lower_text,ticker
0,12.01.2013 18:48,"В четверг, 10 января, Магнит опубликовал предв...","в четверг, 10 января, магнит опубликовал предв...",mgnt
1,14.01.2013 18:56,Крупный портфельный инвестор в Россию — фонд O...,крупный портфельный инвестор в россию — фонд o...,mgnt
2,18.01.2013 22:22,"Ценовой шок, МММщики налетай, МММагнит продается","ценовой шок, мммщики налетай, мммагнит продается",mgnt
3,19.01.2013 14:59,"Хммм, Магнит будет расти и дальше. Нет ему ко...","хммм, магнит будет расти и дальше. нет ему ко...",mgnt
4,20.01.2013 01:09,"Резкого роста не будет,телегу медленно тянут в...","резкого роста не будет,телегу медленно тянут в...",mgnt


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436042 entries, 0 to 436041
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time        436042 non-null  object
 1   text        436042 non-null  object
 2   lower_text  436042 non-null  object
 3   ticker      436042 non-null  object
dtypes: object(4)
memory usage: 13.3+ MB


In [5]:
df.shape

(436042, 4)

In [15]:
df.dtypes

time          object
text          object
lower_text    object
ticker        object
dtype: object

In [17]:
df['date'] = pd.to_datetime(df.time)
df.head()

,time,text,lower_text,ticker,date
0,12.01.2013 18:48,"В четверг, 10 января, Магнит опубликовал предв...","в четверг, 10 января, магнит опубликовал предв...",mgnt,2013-12-01 18:48:00
1,14.01.2013 18:56,Крупный портфельный инвестор в Россию — фонд O...,крупный портфельный инвестор в россию — фонд o...,mgnt,2013-01-14 18:56:00
2,18.01.2013 22:22,"Ценовой шок, МММщики налетай, МММагнит продается","ценовой шок, мммщики налетай, мммагнит продается",mgnt,2013-01-18 22:22:00
3,19.01.2013 14:59,"Хммм, Магнит будет расти и дальше. Нет ему ко...","хммм, магнит будет расти и дальше. нет ему ко...",mgnt,2013-01-19 14:59:00
4,20.01.2013 01:09,"Резкого роста не будет,телегу медленно тянут в...","резкого роста не будет,телегу медленно тянут в...",mgnt,2013-01-20 01:09:00


In [21]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df.head()

,time,text,lower_text,ticker,date,day,month,year
0,12.01.2013 18:48,"В четверг, 10 января, Магнит опубликовал предв...","в четверг, 10 января, магнит опубликовал предв...",mgnt,2013-12-01 18:48:00,1,12,2013
1,14.01.2013 18:56,Крупный портфельный инвестор в Россию — фонд O...,крупный портфельный инвестор в россию — фонд o...,mgnt,2013-01-14 18:56:00,14,1,2013
2,18.01.2013 22:22,"Ценовой шок, МММщики налетай, МММагнит продается","ценовой шок, мммщики налетай, мммагнит продается",mgnt,2013-01-18 22:22:00,18,1,2013
3,19.01.2013 14:59,"Хммм, Магнит будет расти и дальше. Нет ему ко...","хммм, магнит будет расти и дальше. нет ему ко...",mgnt,2013-01-19 14:59:00,19,1,2013
4,20.01.2013 01:09,"Резкого роста не будет,телегу медленно тянут в...","резкого роста не будет,телегу медленно тянут в...",mgnt,2013-01-20 01:09:00,20,1,2013


In [32]:
df['hour'] = df['dt'].dt.hour

In [22]:
df1 = df[(df.month < 10) & (df.day < 10)]
df2 = df[(df.month > 9) & (df.day > 9)]
df3 = df[(df.month < 10) & (df.day > 9)]
df4 = df[(df.month > 9) & (df.day < 10)]
df1.head()

,time,text,lower_text,ticker,date,day,month,year
14,01.02.2013 17:58,папирку Магнита на ветке Сбера стали пиарить а...,папирку магнита на ветке сбера стали пиарить а...,mgnt,2013-01-02 17:58:00,2,1,2013
15,01.02.2013 18:00,"Имхо папирку только форейнеры покупают, которы...","имхо папирку только форейнеры покупают, которы...",mgnt,2013-01-02 18:00:00,2,1,2013
16,01.02.2013 21:30,"Магнит экономкласс, но это 95% населения все ...","магнит экономкласс, но это 95% населения все ...",mgnt,2013-01-02 21:30:00,2,1,2013
17,02.02.2013 14:42,Угу. Только оно начинает придушивать некоторых...,угу. только оно начинает придушивать некоторых...,mgnt,2013-02-02 14:42:00,2,2,2013
18,06.02.2013 04:23,МУДРЫЙ НЕУД 04.02.2013 22:40 бери неуд =бери...,мудрый неуд 04.02.2013 22:40 бери неуд =бери...,mgnt,2013-06-02 04:23:00,2,6,2013


In [23]:
df1['dt'] = df1['date']
df2['dt'] = df2['date']
df3['dt'] = df3['date']
df4['dt'] = df4['date']


df1['date'] = df1['year'].astype(str) + '0' + df1['month'].astype(str) + '0' + df1['day'].astype(str)

df2['date'] = df2['year'].astype(str) + df2['month'].astype(str) + df2['day'].astype(str)

df3['date'] = df3['year'].astype(str) + '0' + df3['month'].astype(str) + df3['day'].astype(str)

df4['date'] = df4['year'].astype(str) + df4['month'].astype(str) + '0' + df4['day'].astype(str)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [24]:
df = pd.concat([df1, df2, df3, df4])
print(df.shape)
df.head()

(436042, 9)


,time,text,lower_text,ticker,date,day,month,year,dt
14,01.02.2013 17:58,папирку Магнита на ветке Сбера стали пиарить а...,папирку магнита на ветке сбера стали пиарить а...,mgnt,20130102,2,1,2013,2013-01-02 17:58:00
15,01.02.2013 18:00,"Имхо папирку только форейнеры покупают, которы...","имхо папирку только форейнеры покупают, которы...",mgnt,20130102,2,1,2013,2013-01-02 18:00:00
16,01.02.2013 21:30,"Магнит экономкласс, но это 95% населения все ...","магнит экономкласс, но это 95% населения все ...",mgnt,20130102,2,1,2013,2013-01-02 21:30:00
17,02.02.2013 14:42,Угу. Только оно начинает придушивать некоторых...,угу. только оно начинает придушивать некоторых...,mgnt,20130202,2,2,2013,2013-02-02 14:42:00
18,06.02.2013 04:23,МУДРЫЙ НЕУД 04.02.2013 22:40 бери неуд =бери...,мудрый неуд 04.02.2013 22:40 бери неуд =бери...,mgnt,20130602,2,6,2013,2013-06-02 04:23:00


In [25]:
df.date = df.date.astype(float)
df.describe()

,date,day,month,year
count,4.360420e+05,436042.000000,436042.000000,436042.000000
mean,2.016946e+07,16.054644,6.487620,2016.879679
std,2.098666e+04,8.706896,3.515336,2.100311
min,2.013010e+07,1.000000,1.000000,2013.000000
25%,2.015101e+07,9.000000,3.000000,2015.000000
50%,2.017053e+07,16.000000,6.000000,2017.000000
75%,2.019012e+07,24.000000,10.000000,2019.000000
max,2.021120e+07,31.000000,12.000000,2021.000000


In [26]:
df.head()

,time,text,lower_text,ticker,date,day,month,year,dt
14,01.02.2013 17:58,папирку Магнита на ветке Сбера стали пиарить а...,папирку магнита на ветке сбера стали пиарить а...,mgnt,20130102.0,2,1,2013,2013-01-02 17:58:00
15,01.02.2013 18:00,"Имхо папирку только форейнеры покупают, которы...","имхо папирку только форейнеры покупают, которы...",mgnt,20130102.0,2,1,2013,2013-01-02 18:00:00
16,01.02.2013 21:30,"Магнит экономкласс, но это 95% населения все ...","магнит экономкласс, но это 95% населения все ...",mgnt,20130102.0,2,1,2013,2013-01-02 21:30:00
17,02.02.2013 14:42,Угу. Только оно начинает придушивать некоторых...,угу. только оно начинает придушивать некоторых...,mgnt,20130202.0,2,2,2013,2013-02-02 14:42:00
18,06.02.2013 04:23,МУДРЫЙ НЕУД 04.02.2013 22:40 бери неуд =бери...,мудрый неуд 04.02.2013 22:40 бери неуд =бери...,mgnt,20130602.0,2,6,2013,2013-06-02 04:23:00


In [36]:
df.groupby('hour').text.count().sort_values()

hour
4       539
3       647
5       803
2      1216
6      1660
1      2427
0      2927
7      3407
8      7125
23    15535
9     16322
22    19481
21    21110
20    22182
19    25825
10    27686
14    32146
13    32580
15    33059
16    33144
12    33515
11    33765
18    34241
17    34700
Name: text, dtype: int64

In [39]:
df.head()

,time,text,lower_text,ticker,date,day,month,year,dt,hour
14,01.02.2013 17:58,папирку Магнита на ветке Сбера стали пиарить а...,папирку магнита на ветке сбера стали пиарить а...,mgnt,20130102.0,2,1,2013,2013-01-02 17:58:00,17
15,01.02.2013 18:00,"Имхо папирку только форейнеры покупают, которы...","имхо папирку только форейнеры покупают, которы...",mgnt,20130102.0,2,1,2013,2013-01-02 18:00:00,18
16,01.02.2013 21:30,"Магнит экономкласс, но это 95% населения все ...","магнит экономкласс, но это 95% населения все ...",mgnt,20130102.0,2,1,2013,2013-01-02 21:30:00,21
17,02.02.2013 14:42,Угу. Только оно начинает придушивать некоторых...,угу. только оно начинает придушивать некоторых...,mgnt,20130202.0,2,2,2013,2013-02-02 14:42:00,14
18,06.02.2013 04:23,МУДРЫЙ НЕУД 04.02.2013 22:40 бери неуд =бери...,мудрый неуд 04.02.2013 22:40 бери неуд =бери...,mgnt,20130602.0,2,6,2013,2013-06-02 04:23:00,4


In [41]:
df.to_csv('/is/texts/mfd/mfd_text_lower_tic_all_flag_all_dt.csv', sep='|', index=False)

In [4]:
df = pd.read_csv('/is/texts/mfd/mfd_text_lower_tic_all_flag_all_dt.csv', sep='|')
df.head()

,time,text,lower_text,ticker,date,day,month,year,dt,hour
0,01.02.2013 17:58,папирку Магнита на ветке Сбера стали пиарить а...,папирку магнита на ветке сбера стали пиарить а...,mgnt,20130102.0,2,1,2013,2013-01-02 17:58:00,17
1,01.02.2013 18:00,"Имхо папирку только форейнеры покупают, которы...","имхо папирку только форейнеры покупают, которы...",mgnt,20130102.0,2,1,2013,2013-01-02 18:00:00,18
2,01.02.2013 21:30,"Магнит экономкласс, но это 95% населения все ...","магнит экономкласс, но это 95% населения все ...",mgnt,20130102.0,2,1,2013,2013-01-02 21:30:00,21
3,02.02.2013 14:42,Угу. Только оно начинает придушивать некоторых...,угу. только оно начинает придушивать некоторых...,mgnt,20130202.0,2,2,2013,2013-02-02 14:42:00,14
4,06.02.2013 04:23,МУДРЫЙ НЕУД 04.02.2013 22:40 бери неуд =бери...,мудрый неуд 04.02.2013 22:40 бери неуд =бери...,mgnt,20130602.0,2,6,2013,2013-06-02 04:23:00,4


In [6]:
df.ticker.unique()

array(['mgnt', 'mtss', 'gmkn', 'sngs', 'sber', 'plzl', 'chmf', 'nvtk',
       'moex', 'afks', 'yndx', 'rsti', 'urka', 'upro', 'trnf', 'tgka',
       'hydr', 'trmk', 'rolo', 'rtkm', 'rual', 'irkt', 'sibn', 'dasb',
       'gche', 'akrn', 'arsa', 'aptk', 'amez', 'phor', 'prfn', 'kbtk',
       'pikk', 'nmtp', 'nknc', 'lsrg', 'msng', 'lnta', 'gtlc', 'yakg',
       'vtb', 'krkn', 'rosn', 'selg', 'gaza', 'fees', 'tatn', 'svav',
       'qiwi', 'rlmn', 'lkoh', 'lsng', 'iskj', 'obuv', 'aflt', 'magn',
       'gazp', 'mtlr', 'alrs', 'rasp', 'tanl'], dtype=object)

In [18]:
try:
    1/0
except: 
    Exception as e:
        print(e)

SyntaxError: invalid syntax (<ipython-input-18-391f96b6dd2b>, line 4)

In [23]:
import sys
counter_except = 0
counter_try = 0
for i in range(10):
    try:
        print(i/0)
        counter_try += 1
    except:
        print("Unexpected error:", sys.exc_info()[0])
        counter_except += 1
print(counter_try, counter_except)

Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
Unexpected error: <class 'ZeroDivisionError'>
0 10


### Аутпут для дат для цен

In [38]:
df.date.to_csv('/is/texts/mfd/news_dates_mfd.csv', sep='|', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [ ]:
df.ticker.replace('vtb', 'vtbr', inplace=True)

In [ ]:
#
есть цены но нет в тг
+trnf 
+tgka
+sibn


не было цен и не было в тг, где + там удалось скачать цены
-urka
+rolo
+irkt
+dasb
+akrn
+arsa
+prfn
+kbtk
+msng
+gtlc
+yakg
+krkn
+selg
+gaza
+fees
+svav
-obuv

In [ ]:
Далее находим изменение цен для этих компаний
джойним их 2020 год будет без до дат